In [5]:
import pandas as pd
import numpy as np
from ord_data_load import ORD_PATH, ORD_REPO_PATH

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#to disable warnings
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

from glob import glob
import os
import multiprocessing as mp
import gzip
from google import protobuf

from ord_schema import message_helpers
from ord_schema.proto import dataset_pb2
from ord_schema.proto import reaction_pb2

from time import time
from ord_data_load import load_dataset, filter_uspto_filenames

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# all_reactions = pd.read_pickle(f"{ORD_PATH}/all_reactions.pkl")

#TODO treat those datasets separately

# Datasets of interest

### 1) Pd CAT REACTIONS DATASETS (500-5k cmpds, Good quality, quantitative)

750 Buchwald-Hartwig reactions generated from AstraZeneca ELN which is used in yield prediction work
*"Graph Neural Networks for Predicting Chemical Reaction Performance"*
[https://doi.org/10.26434/chemrxiv.14589498.v2](https://doi.org/10.26434/chemrxiv.14589498.v2)
[ord_dataset-00005539a1e04c809a9a78647bea649c](https://open-reaction-database.org/client/search?dataset_ids=ord_dataset-00005539a1e04c809a9a78647bea649c)


1536 reactions using HTS with Pd cat. Good quality quantitative data.
*"Nanomole-scale high-throughput chemistry for the synthesis of complex molecules"*
[https://doi.org/10.1126/science.1259203](https://doi.org/10.1126/science.1259203)
[ord_dataset-7d8f5fd922d4497d91cb81489b052746](https://open-reaction-database.org/client/search?dataset_ids=ord_dataset-7d8f5fd922d4497d91cb81489b052746)

5760 Suzuki reactions. Quantitative data. Single scaffold [6-quin]-[4-benzIm]
*A platform for automated nanomole-scale reaction screening and micromole-scale synthesis in flow*
[https://doi.org/10.1126/science.aap9112](https://doi.org/10.1126/science.aap9112)
```ord_dataset-68cb8b4b2b384e3d85b5b1efae58b203```

376 Suzuki reactions. Quantitative data.
Same as scaffold as ```ord_dataset-68cb8b4b2b384e3d85b5b1efae58b203```
[6-quin]-[4-benzIm], but with multiple outcomes as byproducts.
```ord_dataset-eeba974d3c284aed86d1c1d442260a1e```


4312 C-N cross-coupling reactions. Quantitative data. Single scaffold [Ph-NH-Ph].
Different heterocycles - B, Si etc.
*"Predicting reaction performance in C–N cross-coupling using machine learning"*
[https://doi.org/10.1126/science.aar5169](https://doi.org/10.1126/science.aar5169)
```ord_dataset-46ff9a32d9e04016b9380b1b1ef949c3```


288 Buchwald reactions. Quantitative data. Single reaction:
[3-Br-py + 4-Ph-THPyrazine]
```ord_dataset-cbcc4048add7468e850b6ec42549c70d```


450 Pd/Ni cat reactions. Might be usable. Single Scaffold [Ph-Ph]
*"Linking Mechanistic Analysis of Catalytic Reactivity Cliffs to Ligand Classification"*
[https://doi.org/10.26434/chemrxiv.14388557.v1](https://doi.org/10.26434/chemrxiv.14388557.v1)
```ord_dataset-3b5db90e337942ea886b8f5bc5e3aa72```

### 2) 500k REACTIONS, NO CONDITIONS, CLEANED SINGLE DATASET.

500k reactions. Split to train and test. Different reaction classes. No conditions
*"A graph-convolutional neural network model for the prediction of chemical reactivity"*
[https://doi.org/10.1039/C8SC04228D](https://doi.org/10.1039/C8SC04228D)
```
ord_dataset-de0979205c84441190feef587fef8d6d
ord_dataset-488402f6ec0d441ca2f7d6fabea7c220
ord_dataset-5481550056a14935b76e031fb94b88be
```

### 3) USPTO DATABASE ~1.7kk reactions. Needs cleaning. Raw data
Non-zero "patent" field


### 4) everything else - Low quality or small datasets

1728 low variability data.
*Nano CN PhotoChemistry Informers Library*
```ord_dataset-ac78456835404910b3a4c840248b6ac9```


1728 + 256 quantitative data. Imidazole arylation. Single reaction.
https://doi.org/10.1038/s41586-021-03213-y
```ord_dataset-d26118acda314269becc35db5c22dc59```
```ord_dataset-0c75d67751634f0594b24b9f498b77c2```




In [7]:
# uspto_df = all_reactions.loc[~all_reactions.patent.isna()]
# uspto_df

In [8]:
# uspto_filenames = []
# N = len(glob(f'{ORD_REPO_PATH}/data/*/*.pb.gz'))
#
# start = time()
# for i, pb in enumerate(glob(f'{ORD_REPO_PATH}/data/*/*.pb.gz'), 1):
#     print(f"{i:3d} / {N}: parsing dataset {time() - start:.1f}s", end="\r")
#     dataset = load_dataset(pb)
#     if "uspto" in dataset.name:
#         uspto_filenames.append(pb)
#
# len(uspto_filenames)

print("515 / 515: parsing dataset 134.3s")

515 / 515: parsing dataset 134.3s


In [9]:
%%time
n_cores = 24

with mp.Pool(n_cores) as p:
    uspto_filenames = p.map(filter_uspto_filenames, glob(f'{ORD_REPO_PATH}/data/*/*.pb.gz'))

pd.Series(uspto_filenames)

CPU times: total: 281 ms
Wall time: 28.1 s


0                                                   None
1      ./ord-data/data\01\ord_dataset-018fd0e1351f4fd...
2      ./ord-data/data\01\ord_dataset-01dbb772c5e2491...
3      ./ord-data/data\02\ord_dataset-02ee22616630481...
4      ./ord-data/data\03\ord_dataset-0387783899c642a...
                             ...                        
510    ./ord-data/data\fd\ord_dataset-fdef1f30cad6443...
511    ./ord-data/data\fe\ord_dataset-fe016e2f90e741a...
512    ./ord-data/data\fe\ord_dataset-fea3ada7aaad45a...
513    ./ord-data/data\ff\ord_dataset-ff0bcb6c2300494...
514    ./ord-data/data\ff\ord_dataset-ffbef48837674f3...
Length: 515, dtype: object

In [10]:
# save results
uspto_files = pd.Series(uspto_filenames).dropna() #.apply(lambda x: os.path.basename(x).split(".")[0])
uspto_files.to_csv(f"{ORD_PATH}/uspto_files.csv", index=False, header=False)

In [11]:
uspto_files = pd.read_csv(f"{ORD_PATH}/uspto_files.csv").squeeze()

pb = uspto_files.sample().iat[0]
pb

'./ord-data/data\\2d\\ord_dataset-2d6edb8ffd434003bb508360153bd9bb.pb.gz'

In [12]:
dataset = load_dataset(pb)

In [13]:
for field, value in dataset.ListFields():
    print(field.name)

# print(dataset.description)
# print(len(dataset.reactions))

name
description
reactions
dataset_id


In [14]:
dataset.reactions[2].inputs

{'m2': components {
  identifiers {
    type: NAME
    value: "Pd/C"
  }
  identifiers {
    type: SMILES
    value: "[Pd]"
  }
  identifiers {
    type: INCHI
    value: "InChI=1S/Pd"
  }
  amount {
    mass {
      value: 1.2000000476837158
      units: GRAM
    }
  }
  reaction_role: CATALYST
}
, 'm1_m3': components {
  identifiers {
    type: NAME
    value: "1,3-bis(2,7,9-triphenyl-1,4,5,6-tetrahydropyrido[3,2-h]quinazolin-4-yl)benzene"
  }
  identifiers {
    type: SMILES
    value: "C1(=CC=CC=C1)C=1NC=2C3=C(CCC2C(N1)C1=CC(=CC=C1)C1N=C(NC=2C4=C(CCC12)C(=CC(=N4)C4=CC=CC=C4)C4=CC=CC=C4)C4=CC=CC=C4)C(=CC(=N3)C3=CC=CC=C3)C3=CC=CC=C3"
  }
  identifiers {
    type: INCHI
    value: "InChI=1S/C64H48N6/c1-7-20-41(21-8-1)53-39-55(43-24-11-3-12-25-43)65-59-49(53)34-36-51-57(67-63(69-61(51)59)45-28-15-5-16-29-45)47-32-19-33-48(38-47)58-52-37-35-50-54(42-22-9-2-10-23-42)40-56(44-26-13-4-14-27-44)66-60(50)62(52)70-64(68-58)46-30-17-6-18-31-46/h1-33,38-40,57-58H,34-37H2,(H,67,69)(H,68,70)"
  }

In [15]:
dataset.reactions[4].outcomes[0].products

[identifiers {
  type: NAME
  value: "3-(tritylthio)propanoic acid"
}
identifiers {
  type: NAME
  value: "3-(tritylthio) propanoic acid"
}
identifiers {
  type: SMILES
  value: "C(C1=CC=CC=C1)(C1=CC=CC=C1)(C1=CC=CC=C1)SCCC(=O)O"
}
identifiers {
  type: INCHI
  value: "InChI=1S/C22H20O2S/c23-21(24)16-17-25-22(18-10-4-1-5-11-18,19-12-6-2-7-13-19)20-14-8-3-9-15-20/h1-15H,16-17H2,(H,23,24)"
}
measurements {
  type: AMOUNT
  details: "MASS"
  amount {
    mass {
      value: 9.859999656677246
      units: GRAM
    }
  }
}
measurements {
  type: YIELD
  details: "CALCULATEDPERCENTYIELD"
  percentage {
    value: 95.69999694824219
  }
}
reaction_role: PRODUCT
]

In [16]:
dataset.reactions[2].DESCRIPTOR.fields

<MessageFields sequence>

In [17]:
%%time
message_helpers.message_to_row(dataset)

CPU times: total: 6.98 s
Wall time: 6.98 s


{'name': 'uspto-grants-2013_07',
 'description': 'CML filenames: I20130702.xml,I20130709.xml,I20130716.xml,I20130723.xml,I20130730.xml',
 'reactions[0].identifiers[0].type': 'REACTION_CXSMILES',
 'reactions[0].identifiers[0].value': '[NH2:1][C:2]1[CH:10]=[CH:9][C:5]([CH2:6][C:7]#[N:8])=[CH:4][CH:3]=1.N1[CH:16]=[CH:15][CH:14]=[CH:13][CH:12]=1.[Cl-].[OH2:18]>>[C:7]([CH2:6][C:5]1[CH:9]=[CH:10][C:2]([NH:1][C:12]([CH:13]2[CH:4]([CH:5]([CH3:9])[CH3:6])[CH2:3][CH2:2][CH:15]([CH3:16])[CH2:14]2)=[O:18])=[CH:3][CH:4]=1)#[N:8]',
 'reactions[0].identifiers[0].is_mapped': True,
 'reactions[0].inputs["m3_m4"].components[0].identifiers[0].type': 'NAME',
 'reactions[0].inputs["m3_m4"].components[0].identifiers[0].value': 'p-menthane-3-carbon',
 'reactions[0].inputs["m3_m4"].components[0].amount.mass.value': 10.0,
 'reactions[0].inputs["m3_m4"].components[0].amount.mass.units': 'GRAM',
 'reactions[0].inputs["m3_m4"].components[0].reaction_role': 'REACTANT',
 'reactions[0].inputs["m3_m4"].components[1].

In [18]:
%%time
rxn_msg_list = message_helpers.find_submessages(dataset, reaction_pb2.Reaction)
rxn_msg_list[50]

CPU times: total: 0 ns
Wall time: 4.94 ms


identifiers {
  type: REACTION_CXSMILES
  value: "Br[C:2]1[CH:7]=[CH:6][C:5]([OH:8])=[CH:4][C:3]=1[F:9].[F:10][C:11]1[CH:12]=[C:13](B(O)O)[CH:14]=[C:15]([F:17])[CH:16]=1.C(=O)([O-])[O-].[K+].[K+].CC(O)C>Cl[Pd](Cl)([P](C1C=CC=CC=1)(C1C=CC=CC=1)C1C=CC=CC=1)[P](C1C=CC=CC=1)(C1C=CC=CC=1)C1C=CC=CC=1.C1(C)C=CC=CC=1.O>[OH:8][C:5]1[CH:6]=[CH:7][C:2]([C:13]2[CH:12]=[C:11]([F:10])[CH:16]=[C:15]([F:17])[CH:14]=2)=[C:3]([F:9])[CH:4]=1 |f:2.3.4,^1:33,52|"
  is_mapped: true
}
inputs {
  key: "m1_m2_m3_m5_m4"
  value {
    components {
      identifiers {
        type: NAME
        value: "4-Bromo-3-fluorophenol"
      }
      identifiers {
        type: SMILES
        value: "BrC1=C(C=C(C=C1)O)F"
      }
      identifiers {
        type: INCHI
        value: "InChI=1S/C6H4BrFO/c7-5-2-1-4(9)3-6(5)8/h1-3,9H"
      }
      amount {
        mass {
          value: 50.0
          units: GRAM
        }
      }
      reaction_role: REACTANT
    }
    components {
      identifiers {
        type: NAME
    

In [19]:
cmpds = message_helpers.find_submessages(rxn_msg_list[50], reaction_pb2.Compound)
cmpds[1].identifiers[1]

type: SMILES
value: "BrC1=C(C=C(C=C1)O)F"

In [20]:
%%time
from time import time
reactants = np.empty((80000, 2), dtype='<U512')
# reactant_smiles = np.empty((80000, 2), dtype='U256')
# reactant_smiles = np.char.array([""])
reactants_df = pd.DataFrame(columns=['name', 'smiles'])

start = time()
pb = glob(f'{ORD_REPO_PATH}/data/*/*c3c1091f873b4f40827973a6f1f9b685.pb.gz')[0]
# dataset = message_helpers.load_message(pb, dataset_pb2.Dataset)
dataset = load_dataset(pb)
print(f"Dataset {dataset.name} loaded in {time() - start:.2f}s", )

start = time()
idx = 0

for rxn in message_helpers.find_submessages(dataset, reaction_pb2.Reaction):
    cmpds = message_helpers.find_submessages(rxn, reaction_pb2.Compound)
    for cmpd in cmpds:
        if cmpd.reaction_role == reaction_pb2.ReactionRole.REACTANT:
            name = message_helpers.get_compound_identifier(cmpd, reaction_pb2.CompoundIdentifier.NAME)
            smiles = message_helpers.get_compound_identifier(cmpd, reaction_pb2.CompoundIdentifier.SMILES)
            # print(name)
            # reactants = np.append(reactants, [[name, smiles]], axis=0)
            # reactants[idx] = [name, smiles]
            reactants[idx, 0] = name
            reactants[idx, 1] = smiles
            # reactants_df.loc[len(reactants_df), reactants_df.columns] = name, smiles
            idx += 1

print(f"Dataset {dataset.name} parsed in {time() - start:.2f}s", )

print(reactants.shape)
print("Total reactions parsed", len(dataset.reactions))
reactants

Dataset uspto-grants-2014_09 loaded in 0.98s
Dataset uspto-grants-2014_09 parsed in 3.32s
(80000, 2)
Total reactions parsed 17639
CPU times: total: 4.3 s
Wall time: 4.3 s


array([['Hydrochloric acid', 'Cl'],
       ['3-amino-2-naphthol', 'NC=1C(=CC2=CC=CC=C2C1)O'],
       ['sodium carbonate', 'C([O-])([O-])=O.[Na+].[Na+]'],
       ...,
       ['', ''],
       ['', ''],
       ['', '']], dtype='<U512')

In [21]:
reactants_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    0 non-null      object
 1   smiles  0 non-null      object
dtypes: object(2)
memory usage: 0.0+ bytes


In [22]:
reactants_df = pd.DataFrame(columns=['name', 'smiles'])
reactants_df.loc[0, reactants_df.columns] = name, smiles
reactants_df.head()

,name,smiles
0,trifluoroacetic acid,FC(C(=O)O)(F)F


In [23]:
reactants_df

,name,smiles
0,trifluoroacetic acid,FC(C(=O)O)(F)F


In [24]:
idx = 0
reactants[idx] = "ac"
reactants[idx, 1] = "bb"
# reactants[0] = name, smiles
reactants[idx, 1]

'bb'

In [25]:
print(reactants.nbytes)
reactants[np.char.str_len(reactants[:, 0]).argmax()]

327680000


array(['(2S,3S,4S,5R,6R)-benzyl 6-((4-((1R,3aS,5aR,5bR,7aR,11aS,11bR,13aR,13bR)-3a-((2-(1,1-dioxidothiomorpholino)ethyl)amino)-5a,5b,8,8,11a-pentamethyl-1-(prop-1-en-2-yl)-2,3,3a,4,5,5a,5b,6,7,7a,8,11,11a,11b,12,13,13a,13b-octadecahydro-1H-cyclopenta[a]chrysen-9-yl)benzoyl)oxy)-3,4,5-trihydroxytetrahydro-2H-pyran-2-carboxylate',
       'O=S1(CCN(CC1)CCN[C@]12[C@@H]([C@H]3CC[C@@H]4[C@]5(CC=C(C([C@@H]5CC[C@]4([C@@]3(CC1)C)C)(C)C)C1=CC=C(C(=O)O[C@@H]3[C@@H]([C@H]([C@@H]([C@H](O3)C(=O)OCC3=CC=CC=C3)O)O)O)C=C1)C)[C@@H](CC2)C(=C)C)=O'],
      dtype='<U512')